In [16]:
import glob
import pandas as pd
import charset_normalizer

In [6]:
from config.settings import EXTRACT_PATH
from extract.extract import extract_csv

In [ ]:
files = glob.glob(f'{EXTRACT_PATH}/*.csv')

key_columns = ["Country Name","Country Code","Indicator Name","Indicator Code"]
filtered_files = []

for file in files:
    f = open(file)
    for i in range(10):
        line = f.readline()
        if i == 4:
            if any(e in line for e in key_columns):
                filtered_files.append(file)
    f.close()


In [ ]:
def extract_data_to_df(path: str, key_columns:list[str], skip_line = 0) -> list[pd.DataFrame]:  
    files = glob.glob(f'{path}/*.csv')

    df_list:list[pd.DataFrame] = []

    for file in files:
        with open(file, 'rb') as fil:
            result = charset_normalizer.detect(fil.read())
            print(file)
            print(result)
            
        f = open(file)
        for i in range(10):
            line = f.readline()
            if i == skip_line:
                if any(e in line for e in key_columns):
                    df = extract_csv(file,skip_line)
                    df_list.append(df)
        f.close()
    return df_list

In [ ]:
df = extract_data_to_df(EXTRACT_PATH,key_columns,4)

./data\electricity_access_percent.csv
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
./data\projects_data.csv
{'encoding': 'utf-8', 'language': 'English', 'confidence': 0.917}
./data\mystery.csv
{'encoding': 'UTF-16', 'language': 'Vietnamese', 'confidence': 1.0}
./data\population_data.csv
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
./data\rural_population_percent.csv
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
./data\gdp_data.csv
{'encoding': 'UTF-8-SIG', 'language': 'Finnish', 'confidence': 1.0}


In [18]:
for file in filtered_files:
    with open(file, 'rb') as fil:
        result = charset_normalizer.detect(fil.read())
        print(result)

{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
{'encoding': 'UTF-8-SIG', 'language': 'English', 'confidence': 1.0}
{'encoding': 'UTF-8-SIG', 'language': 'Finnish', 'confidence': 1.0}
